### Import the PMNS matrix with 

In [1]:
from neutrinoValues import get_pmns

# Compute the PMNS matrix using the configured values
U = get_pmns()
print(U.round(2))

[[ 0.82+0.j    0.55+0.j   -0.12+0.09j]
 [-0.31+0.05j  0.62+0.04j  0.72+0.j  ]
 [ 0.47+0.05j -0.56+0.03j  0.68+0.j  ]]


### We can test for unitarity

In [2]:
import numpy as np
import neutrinoValues

# Test Unitarity: U * U_dagger = I
U_dagger = U.conj().T
I_check = np.dot(U, U_dagger)
is_unitary = np.allclose(I_check, np.eye(3))

print(f"1. Unitarity Check (U * U† = I): {'PASSED' if is_unitary else 'FAILED'}")
if not is_unitary:
    print("   Resulting Matrix:\n", np.round(I_check, 4))




1. Unitarity Check (U * U† = I): PASSED


### This can also be computed for inverted ordering 

In [3]:
U_inverse = get_pmns(ordering="inverted")
print(U_inverse.round(2))

[[ 0.82+0.j    0.55+0.j    0.04+0.14j]
 [-0.4 +0.09j  0.55+0.06j  0.72+0.j  ]
 [ 0.38+0.08j -0.62+0.05j  0.68+0.j  ]]


### Their difference comes from $\delta_{CP}$

In [4]:
print((U - U_inverse).round(2))

[[ 0.  +0.j    0.  +0.j   -0.15-0.05j]
 [ 0.09-0.03j  0.06-0.02j  0.  +0.j  ]
 [ 0.09-0.03j  0.06-0.02j  0.  +0.j  ]]


### Optionally set a Majorana phase  


In [5]:
U_majorana_phase = get_pmns(alpha=1, beta=1)
print((U_majorana_phase).round(2))   

[[ 0.82+0.j    0.48+0.26j -0.15+0.02j]
 [-0.31+0.05j  0.52+0.33j  0.63+0.35j]
 [ 0.47+0.05j -0.51-0.24j  0.59+0.32j]]


### Now, for consistency, we check against $V_{MNS}$ as given in Gilad Perez and Lisa Randall JHEP01(2009)077 

Setting the possible Majorana phases to zero
for simplicity,

$$
V_{\mathrm{MNS}} =
\begin{pmatrix}
1 & 0 & 0\\
0 & C_{23} & S_{23}\\
0 & -S_{23} & C_{23}
\end{pmatrix}
\begin{pmatrix}
C_{13} & 0 & S_{13} e^{-i\delta_{CP}}\\
0 & 1 & 0\\
- S_{13} e^{i\delta_{CP}} & 0 & C_{13}
\end{pmatrix}
\begin{pmatrix}
C_{12} & S_{12} & 0\\
- S_{12} & C_{12} & 0\\
0 & 0 & 1
\end{pmatrix}
$$


In [6]:

def pmns_matrix(t12, t13, t23, deltacp):
    """
    Build the PMNS matrix V_MNS = R23 @ U13(delta_CP) @ R12
    from mixing angles t12, t13, t23 (radians) and CP phase deltacp (radians).
    """
    c12, s12 = np.cos(t12), np.sin(t12)
    c13, s13 = np.cos(t13), np.sin(t13)
    c23, s23 = np.cos(t23), np.sin(t23)

    e_minus = np.exp(-1j * deltacp)
    e_plus  = np.exp( 1j * deltacp)

    # First block: rotation in (2,3)
    R23 = np.array([
        [1,   0,   0],
        [0,  c23,  s23],
        [0, -s23,  c23]
    ], dtype=complex)

    # Second block: (1,3) with CP phase
    U13 = np.array([
        [ c13,               0,  s13 * e_minus],
        [   0,               1,              0],
        [-s13 * e_plus,      0,           c13]
    ], dtype=complex)

    # Third block: rotation in (1,2)
    R12 = np.array([
        [ c12,  s12, 0],
        [-s12,  c12, 0],
        [   0,    0, 1]
    ], dtype=complex)

    return R23 @ U13 @ R12



In [7]:
# Using radians
t12 = np.deg2rad(33.64708223811001)
t13 = np.deg2rad(8.451338568032302)
t23 = np.deg2rad(46.94956093819177)
deltacp = 1.21 * np.pi # radians

V = pmns_matrix(t12, t13, t23, deltacp)
print(V.round(2))

[[ 0.82+0.j    0.55+0.j   -0.12+0.09j]
 [-0.31+0.05j  0.62+0.04j  0.72+0.j  ]
 [ 0.47+0.05j -0.56+0.03j  0.68+0.j  ]]


In [8]:
same = np.allclose(U, V, rtol=1e-12, atol=1e-12)
print(f"Equal within 10^-12: {same}")

Equal within 10^-12: True
